In [1]:
#!pip install levenshtein

import Levenshtein as lev
import functools
import numpy as np
import pandas as pd
import pickle
import plotly.express as px
import plotly.graph_objects as go

from ttictoc import tic,toc

In [2]:
# Dataframe download

with open("df_feat_added_full.pkl","rb") as f:
    df = pickle.load(f)

In [3]:
# Creating a dictionary, key: sentences count, values: strings of the same num of sentences.
# Levensthein formula will be calculated for each value (list of strings) separately.

class LevenstheinDF:
    
    def __init__(self, df):
        mask = (
            df[['words', 'sentences']].duplicated() == True) & (
            df['sentences'] > 1    
        )
        self.df = df[['name', 'text', 'words', 'sentences', 'verbs_ratio', 'nouns_ratio']][mask]
        self.median_dict = self.get_lev_median()
        self.duplicates = set(self.df['text'].loc[self.df['text'].duplicated(keep=False) == True])
        
    def get_lev_median(self):
        median_dict = dict()
        for i in set(self.df['sentences'].values):
             median_dict[i] = lev.quickmedian(self.df['text'][self.df['sentences'] == i])
        return median_dict 
     
    # Adding median value to the existing dataframe
    def add_to_df(self, x):
        for key, value in self.median_dict.items():
            if x == key:
                return value
    
    # Checking for duplicates and getting ratio
    def check_duplicates(self, x):
        if x[1] in self.duplicates:
            return 0
        else:
            return lev.ratio(x[0], x[1])
            
    # Final ratio calculation    
    def get_ratio(self):
        tic()        
        self.df['median_str'] = self.df['sentences'].apply(self.add_to_df)
        self.df['lev_zip'] = list(zip(self.df['median_str'].values, self.df['text'].values))
        self.df['lev_ratio'] = self.df['lev_zip'].apply(self.check_duplicates)
        elapsed = toc()
        print(f'Time elapsed: {elapsed}')
        return self.df   

In [4]:
leven = LevenstheinDF(df)
compare_df = leven.get_ratio()

Time elapsed: 9.827430000000001


In [5]:
compare_df

,name,text,words,sentences,verbs_ratio,nouns_ratio,median_str,lev_zip,lev_ratio
116,10595647.txt,Spowiedź i rachunek sumienia (dla dorosłych)\n...,4925,474,0.11,0.26,Spowiedź i rachunek sumienia (dla dorosłych)\n...,(Spowiedź i rachunek sumienia (dla dorosłych)\...,0.999939
214,11067484.txt,Aktualności » Ogłoszenia » Intencje i nabożeńs...,1520,85,0.09,0.36,Aktaarości » rłwszenia oIntenc ei nabożeńs a »...,(Aktaarości » rłwszenia oIntenc ei nabożeńs a ...,0.742999
219,11083031.txt,Spowiedź i rachunek sumienia (dla dorosłych)\n...,4925,474,0.11,0.26,Spowiedź i rachunek sumienia (dla dorosłych)\n...,(Spowiedź i rachunek sumienia (dla dorosłych)\...,0.999969
225,11109586.txt,"Prawdę powiedziawszy, modlitwa różańcowa na pi...",465,38,0.12,0.24,Szzeo ik ano oda sir i o ...,(Szzeo ik ano oda sir i o...,0.309677
291,11530580.txt,"Informacje\nSzczegóły Utworzono: piątek, 03, k...",4754,292,0.11,0.25,"Informacje\nSzczegóły Utworzono: piątek, 03, k...","(Informacje\nSzczegóły Utworzono: piątek, 03, ...",0.999967
...,...,...,...,...,...,...,...,...,...
28601,99993801.txt,"Św. Jan Paweł II prosił, by modlitwę „Anioł Pa...",564,52,0.11,0.25,Dooei a i aa oo e ii ...,(Dooei a i aa oo e ii ...,0.260586
28606,99995793.txt,Dzieje Wielkich Oczu\nNazwa miejscowości wywod...,454,32,0.07,0.33,Wii a ...,(Wii a ...,0.126627
28607,99996277.txt,„Wychowanie w czasie sporów w kulturowych” - c...,447,29,0.05,0.40,Dzio a ...,(Dzio a ...,0.130550
28610,99998238.txt,W podziękowaniu za pół wieku posługi organisto...,782,55,0.09,0.28,Da oe w a i a zi o ie ...,(Da oe w a i a zi o ie...,0.311045


In [6]:
with open("compare_df.pkl","wb") as f:
    pickle.dump(compare_df, f)

In [7]:
compare_df['lev_ratio'].describe()

count    8379.000000
mean        0.287037
std         0.227713
min         0.084618
25%         0.146610
50%         0.186326
75%         0.326130
max         1.000000
Name: lev_ratio, dtype: float64

In [51]:
def get_similar(compare_df):
    return compare_df[
        (compare_df['lev_ratio'] > 0.4)
    ].sort_values(by=['lev_ratio', 'text'])

def get_different(compare_df):
    return compare_df[
        (compare_df['lev_ratio'] <= 0.4)
    ].sort_values(by=['lev_ratio', 'text'])

In [52]:
get_similar(compare_df)

,name,text,words,sentences,verbs_ratio,nouns_ratio,median_str,lev_zip,lev_ratio
18282,78856726.txt,Nabożeństwo z koronką do Miłosierdzia Bożego i...,796,63,0.03,0.17,Biblaż arstww kz kięga: Ew dolia św. aeeuszi ...,(Biblaż arstww kz kięga: Ew dolia św. aeeuszi ...,0.400447
17052,75300309.txt,Nabożeństwo z koronką do Miłosierdzia Bożego i...,796,63,0.03,0.17,Biblaż arstww kz kięga: Ew dolia św. aeeuszi ...,(Biblaż arstww kz kięga: Ew dolia św. aeeuszi ...,0.400447
24957,94794276.txt,Nabożeństwo z koronką do Miłosierdzia Bożego i...,796,63,0.03,0.17,Biblaż arstww kz kięga: Ew dolia św. aeeuszi ...,(Biblaż arstww kz kięga: Ew dolia św. aeeuszi ...,0.400447
21171,8654013.txt,Nabożeństwo z koronką do Miłosierdzia Bożego i...,796,63,0.03,0.17,Biblaż arstww kz kięga: Ew dolia św. aeeuszi ...,(Biblaż arstww kz kięga: Ew dolia św. aeeuszi ...,0.400447
5544,35268757.txt,Nabożeństwo z koronką do Miłosierdzia Bożego i...,796,63,0.03,0.17,Biblaż arstww kz kięga: Ew dolia św. aeeuszi ...,(Biblaż arstww kz kięga: Ew dolia św. aeeuszi ...,0.400484
...,...,...,...,...,...,...,...,...,...
13259,64030708.txt,"XII NIEDZIELA ZWYKŁA, ROK C, 23. 06. 2019r. Py...",4338,259,0.11,0.25,"XII NIEDZIELA ZWYKŁA, ROK C, 23. 06. 2019r. Py...","(XII NIEDZIELA ZWYKŁA, ROK C, 23. 06. 2019r. P...",1.000000
21075,86292894.txt,"XII NIEDZIELA ZWYKŁA, ROK C, 23. 06. 2019r. Py...",4338,263,0.11,0.25,"XII NIEDZIELA ZWYKŁA, ROK C, 23. 06. 2019r. Py...","(XII NIEDZIELA ZWYKŁA, ROK C, 23. 06. 2019r. P...",1.000000
14835,68986119.txt,ZGROMADZENI NA ŚWIĘTEJ WIECZERZY\nList biskupó...,2480,143,0.10,0.27,ZGROMADZENI NA ŚWIĘTEJ WIECZERZY\nList biskupó...,(ZGROMADZENI NA ŚWIĘTEJ WIECZERZY\nList biskup...,1.000000
26448,973786.txt,ŚW. MARIA MAGDALENA - patronka naszej parafii\...,1585,95,0.11,0.21,ŚW. MARIA MAGDALENA - patronka naszej parafii\...,(ŚW. MARIA MAGDALENA - patronka naszej parafii...,1.000000


In [65]:
get_different(compare_df)

,name,text,words,sentences,verbs_ratio,nouns_ratio,median_str,lev_zip,lev_ratio
24562,9401232.txt,Czym i dla kogo są N.O.? (Niedzielne Oczywisto...,455,36,0.11,0.24,P o ...,(P o ...,0.084618
9910,52343158.txt,Poniedziałek 12 grudnia 2022\n6:00\n+ Siostra ...,545,36,0.05,0.26,P o ...,(P o ...,0.102739
26736,97809564.txt,Data: 03.01.2023\nZamierzenia dydaktyczno – wy...,557,34,0.06,0.38,Wo i ...,(Wo i ...,0.103947
10170,53304492.txt,1. Ponawiamy zaproszenie J.E. Abp Metropolity ...,463,33,0.03,0.20,D ioo ...,(D ioo ...,0.104923
24582,94054701.txt,Bierzmowanie\nWarunki uczestniczenia w przygot...,428,35,0.06,0.34,D i ...,(D i ...,0.107779
...,...,...,...,...,...,...,...,...,...
9815,5205030.txt,Parafia Mszano - Aktualności\nSzczegóły: | Pop...,927,51,0.06,0.26,Parcii szano Op ublnii owzazeo: eoezaiel...,(Parcii szano Op ublnii owzazeo: eoezaie...,0.395694
25937,96593783.txt,Pewnego razu w Niemczech odbyły się rekolekcje...,886,57,0.13,0.20,Aktualności Słowo na niedzielę\nSzczegóły Kate...,(Aktualności Słowo na niedzielę\nSzczegóły Kat...,0.396972
4465,3060020.txt,Wymodlona i wywalczona wolność\nUmiłowani w Pa...,1570,75,0.07,0.29,W orzi IIaw a Trlssi eana aaruu ios yih...,(W orzi IIaw a Trlssi eana aaruu ios yi...,0.397634
18348,79025969.txt,LIST PASTERSKI BISKUPA PŁOCKIEGO PIOTRA LIBERY...,1338,90,0.11,0.23,List iAAłyE SzE para ia Prski EppWojaiuąca o2...,(List iAAłyE SzE para ia Prski EppWojaiuąca o...,0.397991


In [62]:
# It seems that shorter documents with small proper nouns count in our set are most likely to be duplicates.

fig = px.scatter_polar(
    compare_df,
    r=-compare_df['lev_ratio'],
    theta=compare_df['sentences'],
    color=compare_df['words'],
    color_continuous_scale=px.colors.cyclical.HSV,
    template="plotly_dark")
fig.show()

In [63]:
fig = px.histogram(compare_df['lev_ratio'])
fig.show()

In [64]:
compare_df['lev_ratio'].value_counts()

0.984254    44
0.966121    40
0.999939    40
0.729869    37
1.000000    36
            ..
0.309360     1
0.144078     1
0.167545     1
0.133776     1
0.163043     1
Name: lev_ratio, Length: 7140, dtype: int64